In [30]:
import numpy
from matplotlib import pyplot as plt
import logging
%matplotlib inline

Write a code to generate and display a cellular automaton with two changes from the binary CA case:

1) replace the 2-state system with a 3-state system. Thus a cell, instead of being either 0 or 1, can be 0, 1, or 2.

2) define the neighborhood as the cell above and to the left, rather than above, to the left, and to the right. This is so that there are 3^9 = 19683 possible input configurations instead of 3^(3^3) which is greater than 10^{12}. (By 'neighborhood' here we mean the set of cells that, together with a rule, determine a cell's value. In the 2-state example we did in class, the neighborhood for a given cell was the cell above and its nearest neighbors.)

Do two versions: one without use of functions and classes, and one with use of functions and at least one class. See if you can reuse some of Adam Rupe's 2-state CA code. Upload your solutions as a single Jupyter Notebook.

In [31]:
def numberToBase(n: int, b: int) -> list[int]:
    """
    Source: https://stackoverflow.com/a/28666223
    Converts any number to a list of integers corresponding to different digits.
    For example, 15 in decimal would be returned as [1, 5].
    5 in binary would be returned as [1, 0, 1]
    :param n: Input number
    :param b: Base to convert input number n
    :return: List of digits
    """
    if n == 0:
        return [0]
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    return digits[::-1]

In [32]:
def generate_neighborhoods(neighborhood_members: int, num_states: int) -> list[tuple]:
    """
    Generates a list of tuples corresponding to possible neighborhoods
    :param neighborhood_members: Number of elements in each neighborhood. For example, left, above, and right is neighborhood_members=3.
    :param num_states: Number of possible values for each element in a neighborhood. For example, a binary system with 0 and 1 is num_states=2.
    :return: List of tuples. For example, [(0,0,0), (0,0,1), ...]
    """
    neighborhoods: list[tuple] = []

    # loop until the length of neighborhoods exceed neighborhood_members
    decimal_number = 0
    while True:
        number_list: list[int] = numberToBase(decimal_number, num_states)  # gives list of digits like [1,0,1]
        number_string: str = ''.join([str(i) for i in number_list]).zfill(neighborhood_members)  # should be a string with length=neighborhood_members like 0101
        if len(number_string) > neighborhood_members:
            break
        to_append = tuple((int(char) for char in number_string))
        neighborhoods.append(to_append)
        logging.info(number_list, number_string, to_append)
        decimal_number += 1
    return neighborhoods


# Main function with most of code

In [33]:
def main(rule_number: int, length: int, time: int, neighborhood_members: int, num_states: int):
    assert 0 <= rule_number <= 255, "Please choose a value for rule_number between 0 and 255"
    assert length > 0, "Please choose a value for length greater than 0"
    assert time > 0, "Please choose a value for time greater than 0"

# make the initial condition
initial_condition: list[int] = []
for i in range(length):
    initial_condition.append(random.randint(0,1))

# gives list of tuples like [(0,0,0), (0,0,1), ...]
neighborhoods: list[tuple] = generate_neighborhoods(neighborhood_members, num_states)

#
#
# **Tests**

In [34]:
def numberToBase_test() -> list[int]:
    assert numberToBase(15, 10) == [1, 5]
    assert numberToBase(5, 2) == [1, 0, 1]
    assert numberToBase(27, 6) == [4, 3]
    print("numberToBase_test passed")

In [35]:
numberToBase_test()

numberToBase_test passed


In [36]:
def generate_neighborhoods_tests()-> list[tuple]:
    assert type(generate_neighborhoods(3, 2)) == list, "function generate_neighborhood() is not returning lists"
    try:
        assert type(generate_neighborhoods(3, 2)[0]) == tuple, "the elements of function generate_neighborhood() are not all tuples"
    except IndexError:
        raise IndexError("The list returned from function generate_neighborhoods() is empty")
    assert generate_neighborhoods(3, 2) == [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,0), (1,0,1), (1,1,0), (1,1,1)], "The tuples generated by function generate_neighborhoods(3, 2) is not correct"
    assert generate_neighborhoods(4, 2) == [(0,0,0,0), (0,0,0,1), (0,0,1,0), (0,0,1,1), (0,1,0,0), (0,1,0,1), (0,1,1,0), (0,1,1,1), (1,0,0,0), (1,0,0,1), (1,0,1,0), (1,0,1,1), (1,1,0,0), (1,1,0,1), (1,1,1,0), (1,1,1,1)], "The tuples generated by function generate_neighborhoods(4, 2) is not correct"
    assert generate_neighborhoods(2, 3) == [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)], "The tuples generated by function generate_neighborhoods(2, 3) is not correct"
    print("generate_neighborhoods_tests passed")

In [37]:
generate_neighborhoods_tests()

generate_neighborhoods_tests passed
